## Plot stick figure from home video analysis "run.py" outputs 
run.py outputs as of 7/31/2024 version 

- Function 1 - plot 2D 
- Function 2 - plot 3D top down view
- Function 3 - plot 3D straight on view

Can use either pose or world landmarks outputs 

## Import libraries 

In [1]:
import matplotlib.pyplot as plt 
import pandas as pd 
import os 

## Load and review MediaPipe data 

In [2]:
print(os.getcwd())

C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\hva_code\HomeVideoAnalysis\megan_sandbox\visualize_pose_outputs


In [3]:
# set input file and output folder - individidually for now, make loop later 

#adjust input_file 
input_file_full = 'DS_HC_gait_horizontal_left_mediapipe.csv'

#keep these constant 
output_folder = r'..\..\temp\main_branch_outputs\002_visualize\DS_HC\gait_horizontal_left_mediapipe'
print(output_folder)
input_folder = r'..\..\temp\main_branch_outputs\000_run'
print(input_folder)

#merge input folder and file -> input filepath 
input_file_path = os.path.normpath(os.path.join(input_folder, input_file_full))
print(input_file_path)


..\..\temp\main_branch_outputs\002_visualize\DS_HC\gait_horizontal_left_mediapipe
..\..\temp\main_branch_outputs\000_run
..\..\temp\main_branch_outputs\000_run\DS_HC_gait_horizontal_left_mediapipe.csv


In [4]:
df_mp = pd.read_csv(input_file_path)

In [5]:
df_mp.head(n=5)

,Unnamed: 0,X,Y,Z,vis,pres,frame,label
0,0,0.800228,0.099891,-0.633101,0.998995,0.0,0,nose
1,1,0.760156,0.079382,-0.542120,0.998117,0.0,0,left_eye_inner
2,2,0.753662,0.079156,-0.542586,0.997281,0.0,0,left_eye
3,3,0.745315,0.078838,-0.542838,0.998063,0.0,0,left_eye_outer
4,4,0.741882,0.080162,-0.772307,0.998752,0.0,0,right_eye_inner


In [6]:
df_mp.dtypes

Unnamed: 0      int64
X             float64
Y             float64
Z             float64
vis           float64
pres          float64
frame           int64
label          object
dtype: object

In [7]:
df_mp.shape # (rows, columns)

(63195, 8)

In [8]:
# rename columns 
list(df_mp.columns)
df_mp = df_mp.rename(columns = {'Unnamed: 0':'label_num'})
df_mp.head()

,label_num,X,Y,Z,vis,pres,frame,label
0,0,0.800228,0.099891,-0.633101,0.998995,0.0,0,nose
1,1,0.760156,0.079382,-0.542120,0.998117,0.0,0,left_eye_inner
2,2,0.753662,0.079156,-0.542586,0.997281,0.0,0,left_eye
3,3,0.745315,0.078838,-0.542838,0.998063,0.0,0,left_eye_outer
4,4,0.741882,0.080162,-0.772307,0.998752,0.0,0,right_eye_inner


In [9]:
# min and max to standardize plot axes  
x_min = min(df_mp['X'])
x_max = max(df_mp['X'])
y_min = min(df_mp['Y'])
y_max = max(df_mp['Y'])
z_min = min(df_mp['Z'])
z_max = max(df_mp['Z'])

# last frame to run through all frames 
frame_max = max(df_mp['frame'])

## Function to plot all points in 2D with labels 

In [10]:
# current_frame = frame used to filter data, current frame being plotted 
# current_frame_df = pandas data frame, ouptut of mediapipe pose, filtered to only include data frame frame = current_frame
# input file - string, name of .csv file, set in chunks above when loading data 

def plot_pose_single_frame_2D(current_frame, current_frame_df, input_file_full):
    print('current_frame: ' + str(current_frame))
    
    # save x y and label as vector  
    x = current_frame_df[['X']]
    y = current_frame_df[['Y']]
    label = current_frame_df['label']

    # create figure and ax
    fig = plt.figure()
    ax = fig.add_subplot(111)

    #plot scatter plot 
    plt.scatter(current_frame_df['X'], -current_frame_df['Y']) # negative Y to flip image with feet on ground 

    # add text with marker label
    for i, txt in enumerate(label):
        ax.text(current_frame_df.iloc[i]['X'], -current_frame_df.iloc[i]['Y'], txt)

    # axis limits 
    plt.xlim(x_min, x_max)
    plt.ylim(-y_max, -y_min) # flip to match negative y 

    # titles and labels 
    plt.title(input_file_full)
    plt.suptitle("Frame #: " + str(current_frame))

    if "world" in input_file_full:
        plt.xlabel('X (meters)')
        plt.ylabel('Y (meters)')
    else: 
        plt.xlabel('X (pose - confirm w Yoni)')
        plt.ylabel('Y (pose - confirm w Yoni)')
    
    # create file name to save image
        # input file name _ current frame.png
    input_file_no_ext = os.path.splitext(os.path.basename(input_file_full))[0]
    output_file_name = input_file_no_ext + '_' + str(current_frame) + '_2D.png'
    output_file = os.path.normpath(os.path.join(output_folder, output_file_name))

    # save and show figure 
    print('saving: ' + output_file)
    plt.savefig(output_file)
    #plt.show()
    plt.close()
    


## Function to plot 3D with top down view 
confirm - lighter dots = further from camera 

In [11]:
# current_frame = frame used to filter data, current frame being plotted 
# current_frame_df = pandas data frame, ouptut of mediapipe pose, filtered to only include data frame frame = current_frame
# input file - string, name of .csv file, set in chunks above when loading data 

def plot_pose_single_frame_3D_XZ(current_frame, current_frame_df, input_file_full):
    label = current_frame_df['label']

    fig_labels = plt.figure()
    ax = plt.axes(projection = '3d')
    ax.scatter3D(current_frame_df['X'], -current_frame_df['Y'], current_frame_df['Z'])
    plt.xlabel("X (pixels)")
    plt.ylabel("Y (pixels)")

    # add text 
    for i, txt in enumerate(label):
        ax.text(current_frame_df.iloc[i]['X'], -current_frame_df.iloc[i]['Y'], current_frame_df.iloc[i]['Z'], txt)
    
    plt.xlim(x_min, x_max)
    plt.ylim(-y_max, -y_min) # flip to match negative y 

    # titles  
    plt.title(input_file_full)
    plt.suptitle("Frame #: " + str(current_frame))

    # axis labels 
    if "world" in input_file_full:
        plt.xlabel('X (meters)')
        plt.ylabel('Y (meters)')
        plt.zlabel('Z (meters)')
    else: 
        plt.xlabel('X (pose - confirm w Yoni)')
        plt.ylabel('Y (pose - confirm w Yoni)')

    # top down view ----------------------------------------------
    ax.view_init(0, -90, 0)
   
    # create file name to save image
        # input file name _ current frame.png
    input_file_no_ext = os.path.splitext(os.path.basename(input_file_full))[0]
    output_file_name = input_file_no_ext + '_' + str(current_frame) + '_3D_xz.png'
    output_file = os.path.normpath(os.path.join(output_folder, output_file_name))

    # save and show figure 
    plt.savefig(output_file)
    print('saving: ' + output_file)
    plt.close()
    

## Function to plot 3D viewing like frame of camera 
- confirm from top down plots- lighter dots = further from camera
- no labels 

In [12]:
# current_frame = frame used to filter data, current frame being plotted 
# current_frame_df = pandas data frame, ouptut of mediapipe pose, filtered to only include data frame frame = current_frame
# input file - string, name of .csv file, set in chunks above when loading data 

def plot_pose_single_frame_3D_XY(current_frame, current_frame_df, input_file_full):
    label = current_frame_df['label']

    fig_labels = plt.figure()
    ax = plt.axes(projection = '3d')
    ax.scatter3D(current_frame_df['X'], -current_frame_df['Y'], current_frame_df['Z'])
    plt.xlabel("X (pixels)")
    plt.ylabel("Y (pixels)")

    # add text 
    #for i, txt in enumerate(label):
        #ax.text(current_frame_df.iloc[i]['X'], -current_frame_df.iloc[i]['Y'], current_frame_df.iloc[i]['Z'], txt)
    
    plt.xlim(x_min, x_max)
    plt.ylim(-y_max, -y_min) # flip to match negative y 

    # titles  
    plt.title(input_file_full)
    plt.suptitle("Frame #: " + str(current_frame))

    # axis labels 
    if "world" in input_file_full:
        plt.xlabel('X (meters)')
        plt.ylabel('Y (meters)')
        plt.zlabel('Z (meters)')
    else: 
        plt.xlabel('X (pose - confirm w Yoni)')
        plt.ylabel('Y (pose - confirm w Yoni)')

    # camera view ----------------------------------------------
    ax.view_init(100, -90, 0)
   
    # create file name to save image
        # input file name _ current frame.png
    input_file_no_ext = os.path.splitext(os.path.basename(input_file_full))[0]
    output_file_name = input_file_no_ext + '_' + str(current_frame) + '_3D_xy.png'
    output_file = os.path.normpath(os.path.join(output_folder, output_file_name))

    # save and show figure 
    plt.savefig(output_file)
    print('saving: ' + output_file)
    plt.close()
    

### Plot

In [13]:
frames = range(0,frame_max,25) #start, stop, step

for current_frame in frames: 
    # pull rows that correspond to current_frame2
    current_frame_df = df_mp.loc[df_mp['frame'] == current_frame]

    #plots 
    #2D, w labels 
    plot_pose_single_frame_2D(current_frame, current_frame_df, input_file_full)
    # 3D, xz, w labels 
    plot_pose_single_frame_3D_XZ(current_frame, current_frame_df, input_file_full)
    # 3D, xy, w/o labels 
    plot_pose_single_frame_3D_XY(current_frame, current_frame_df, input_file_full)
    

current_frame: 0
saving: ..\..\temp\main_branch_outputs\002_visualize\DS_HC\gait_horizontal_left_mediapipe\DS_HC_gait_horizontal_left_mediapipe_0_2D.png
saving: ..\..\temp\main_branch_outputs\002_visualize\DS_HC\gait_horizontal_left_mediapipe\DS_HC_gait_horizontal_left_mediapipe_0_3D_xz.png
saving: ..\..\temp\main_branch_outputs\002_visualize\DS_HC\gait_horizontal_left_mediapipe\DS_HC_gait_horizontal_left_mediapipe_0_3D_xy.png
current_frame: 25
saving: ..\..\temp\main_branch_outputs\002_visualize\DS_HC\gait_horizontal_left_mediapipe\DS_HC_gait_horizontal_left_mediapipe_25_2D.png
saving: ..\..\temp\main_branch_outputs\002_visualize\DS_HC\gait_horizontal_left_mediapipe\DS_HC_gait_horizontal_left_mediapipe_25_3D_xz.png
saving: ..\..\temp\main_branch_outputs\002_visualize\DS_HC\gait_horizontal_left_mediapipe\DS_HC_gait_horizontal_left_mediapipe_25_3D_xy.png
current_frame: 50
saving: ..\..\temp\main_branch_outputs\002_visualize\DS_HC\gait_horizontal_left_mediapipe\DS_HC_gait_horizontal_left